In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from typing import List, Optional
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException,TimeoutException, StaleElementReferenceException, ElementClickInterceptedException
import requests
from bs4 import BeautifulSoup
from pathlib import Path
from datetime import date
import re

## Scraping Webiste

### Build Driver and Load full HTML page by click button or scroll infinity

In [3]:
class driver_to_crawl_page:
    def __init__(self, 
                 headless: bool = True,
                 url: str ='https://www.cheapcarco.com.au/used-vehicles-rocklea/list',
                 element_by: str = 'xpath',
                 elements_value: str = "//main/*/*/*/*/*/*/*/*/*/div[contains(@class, 'flex flex-col h-full')]"
                 ):
        self.url = url
        self.headless = headless
        self.element_by = element_by
        self.elements_value=elements_value
        
    def build_driver(self) -> webdriver.Chrome:
        chrome_options = Options()
        if self.headless:
            chrome_options.add_argument("--headless=new")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-gpu")
            chrome_options.add_argument("--window-size=1920,1080")
            chrome_options.add_argument("--disable-dev-shm-usage")
            driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=chrome_options
        )
        return driver
    
    def load_full_page (self, timeout: float = 0.5, button_path: str = '/html/body/div[1]/div/main/div/div/div/div/div[2]/button') -> webdriver:
        driver = self.build_driver()
        driver.get(self.url)

        while True:
            try:
                time.sleep(timeout)
                button = driver.find_element(by = self.element_by, value = button_path)
                button.click()
            except NoSuchElementException:
                print('Crawling new products list')
                break

        products = driver.find_elements(by = self.element_by, value = self.elements_value)
        return products
        

In [4]:
driver = driver_to_crawl_page().build_driver()
products = driver_to_crawl_page().load_full_page()


Crawling new products list


### Crawl website by find element

In [5]:
class WebsiteCrawling:

    def __init__(self,
                 product: webdriver,
                 driver: webdriver.Chrome,
                 url: str ='https://www.cheapcarco.com.au/used-vehicles-rocklea/list',
                 elements_by: str = 'xpath'
                 ):
        self.url = url
        self.driver = driver
        self.driver.get(self.url)
        self.elements_by = elements_by
        self.products = product
    
    def get_link (self, path: str, get_attribute_part: str) -> list:
        link = []
        for p in self.products:
            try:
                l = p.find_element(by = self.elements_by, value = path)
                link.append(l.get_attribute(get_attribute_part))
            except NoSuchElementException:
                link.append(None)
                continue

        return link
    def get_text (self, path: str) -> list:
        text = []
        for p in self.products:
            try:
                t = p.find_element(by = self.elements_by, value = path)
                text.append(t.text)
            except NoSuchElementException:
                text.append(None)
                continue
        return text

In [6]:
name = WebsiteCrawling(products,driver).get_text('.//h5')
link_product = WebsiteCrawling(products,driver).get_link('.//a','href')
link_image = WebsiteCrawling(products,driver).get_link('.//a/div/img','src')
price = WebsiteCrawling(products,driver).get_text('.//div[3]/h3')
odo = WebsiteCrawling(products,driver).get_text('.//div[2]/div[1]/span[2]')
cyl = WebsiteCrawling(products,driver).get_text('.//div[2]/div[2]/span[2]')
trans = WebsiteCrawling(products,driver).get_text('.//div[2]/div[3]/span[2]')
fuel = WebsiteCrawling(products,driver).get_text('.//div[4]/span[2]')


In [7]:
now = date.today()
cheapcarco = {
    'name':name,
    'link_product': link_product,
    'link_image': link_image,
    'price':price,
    'odometer':odo,
    'cylinder':cyl,
    'transmission': trans,
    'fuel_type':fuel,
    'last_update': now
    }

cheapcarco = pd.DataFrame(cheapcarco)
cheapcarco

,name,link_product,link_image,price,odometer,cylinder,transmission,fuel_type,last_update
0,2014 BMW 1 Series F20 MY0713 116i Steptronic G...,https://www.cheapcarco.com.au/used-vehicles-ro...,https://radius-cdn.ramp.coxautoinc.com.au/_rad...,"$12,999","140,100 Kms",4 Cylinders,Sports Automatic,Petrol - Premium ULP,2025-10-24
1,2015 Mazda 2 DJ2HAA Genki SKYACTIV-Drive Blue ...,https://www.cheapcarco.com.au/used-vehicles-ro...,https://radius-cdn.ramp.coxautoinc.com.au/_rad...,"$14,999","128,819 Kms",4 Cylinders,Sports Automatic,Petrol - Unleaded,2025-10-24
2,2008 Mazda 3 BK10F2 MY08 Neo Sport Blue 4 Spee...,https://www.cheapcarco.com.au/used-vehicles-ro...,https://radius-cdn.ramp.coxautoinc.com.au/_rad...,"$6,999","233,336 Kms",4 Cylinders,Sports Automatic,Petrol - Unleaded,2025-10-24
3,2014 Mazda 3 BM5236 SP25 SKYACTIV-MT Astina Si...,https://www.cheapcarco.com.au/used-vehicles-ro...,https://radius-cdn.ramp.coxautoinc.com.au/_rad...,"$7,999","248,433 Kms",4 Cylinders,Manual,Petrol - Unleaded,2025-10-24
4,2008 BMW 3 Series E90 MY08 320i Steptronic Mar...,https://www.cheapcarco.com.au/used-vehicles-ro...,https://radius-cdn.ramp.coxautoinc.com.au/_rad...,"$9,999","98,517 Kms",4 Cylinders,Sports Automatic,Petrol - Premium ULP,2025-10-24
...,...,...,...,...,...,...,...,...,...
141,2016 Nissan X-Trail T32 ST X-tronic 4WD Grey 7...,https://www.cheapcarco.com.au/used-vehicles-ro...,https://radius-cdn.ramp.coxautoinc.com.au/_rad...,"$13,999","166,860 Kms",4 Cylinders,Constant Variable,Petrol - Unleaded,2025-10-24
142,2009 BMW X5 E70 MY09 xDrive30d Steptronic Blac...,https://www.cheapcarco.com.au/used-vehicles-ro...,https://radius-cdn.ramp.coxautoinc.com.au/_rad...,"$10,999","212,194 Kms",6 Cylinders,Sports Automatic,Diesel,2025-10-24
143,2015 BMW X5 F15 xDrive30d Grey 8 Speed Sports ...,https://www.cheapcarco.com.au/used-vehicles-ro...,https://radius-cdn.ramp.coxautoinc.com.au/_rad...,"$19,999","142,313 Kms",6 Cylinders,Sports Automatic,Diesel,2025-10-24
144,2013 Subaru XV G4X MY14 2.0i AWD Blue 6 Speed ...,https://www.cheapcarco.com.au/used-vehicles-ro...,https://radius-cdn.ramp.coxautoinc.com.au/_rad...,"$10,999","231,462 Kms",4 Cylinders,Manual,Petrol - Unleaded,2025-10-24


## Wrangling

In [18]:
df = cheapcarco
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          146 non-null    object
 1   link_product  146 non-null    object
 2   link_image    146 non-null    object
 3   price         146 non-null    object
 4   odometer      146 non-null    object
 5   cylinder      146 non-null    object
 6   transmission  146 non-null    object
 7   fuel_type     145 non-null    object
 8   last_update   146 non-null    object
dtypes: object(9)
memory usage: 10.4+ KB


In [ ]:
"""drop duplicates"""
df = df.drop_duplicates()

"""add column year made"""
df['year_made'] = df['name'].str[:4].astype(int)


""" add columm car brand"""
pattern = re.compile(r'^\d{4}\s+([a-zA-z]+)')
df['brand']=df['name'].str.extract(pattern, expand = False)

""" add column car name"""
pattern = re.compile(r'^\d{4}\s+[a-zA-z]+\s([a-zA-z]+)')
df ['name'] = df['name'].str.extract(pattern, expand = False)

""" change price to int"""
df['price']=(df['price'].str.replace(r'[\$,]','',regex=True)).astype(int)

""" change odometer to int"""
df['odometer'] = (df['odometer'].str.replace(r'[\s[Kms,]','',regex=True)).astype(int)

""" change cylinder to int"""
df['cylinder'] = (df['cylinder'].str.replace(r'[\s[Kms]','',regex=True)).astype(int)



AttributeError: Can only use .str accessor with string values!

In [44]:
pattern = re.compile(r'^\d{4}\s+[a-zA-z]+\s([a-zA-z]+)')
df ['name'] = df['name'].str.extract(pattern, expand = False)
df

,name,link_product,link_image,price,odometer,cylinder,transmission,fuel_type,last_update,year_made,brand,odometer (Kms)
0,NaN,https://www.cheapcarco.com.au/used-vehicles-ro...,https://radius-cdn.ramp.coxautoinc.com.au/_rad...,12999,140100,4 Cylinders,Sports Automatic,Petrol - Premium ULP,2025-10-24,2014,BMW,140100
1,NaN,https://www.cheapcarco.com.au/used-vehicles-ro...,https://radius-cdn.ramp.coxautoinc.com.au/_rad...,14999,128819,4 Cylinders,Sports Automatic,Petrol - Unleaded,2025-10-24,2015,Mazda,128819
2,NaN,https://www.cheapcarco.com.au/used-vehicles-ro...,https://radius-cdn.ramp.coxautoinc.com.au/_rad...,6999,233336,4 Cylinders,Sports Automatic,Petrol - Unleaded,2025-10-24,2008,Mazda,233336
3,NaN,https://www.cheapcarco.com.au/used-vehicles-ro...,https://radius-cdn.ramp.coxautoinc.com.au/_rad...,7999,248433,4 Cylinders,Manual,Petrol - Unleaded,2025-10-24,2014,Mazda,248433
4,NaN,https://www.cheapcarco.com.au/used-vehicles-ro...,https://radius-cdn.ramp.coxautoinc.com.au/_rad...,9999,98517,4 Cylinders,Sports Automatic,Petrol - Premium ULP,2025-10-24,2008,BMW,98517
...,...,...,...,...,...,...,...,...,...,...,...,...
141,X,https://www.cheapcarco.com.au/used-vehicles-ro...,https://radius-cdn.ramp.coxautoinc.com.au/_rad...,13999,166860,4 Cylinders,Constant Variable,Petrol - Unleaded,2025-10-24,2016,Nissan,166860
142,X,https://www.cheapcarco.com.au/used-vehicles-ro...,https://radius-cdn.ramp.coxautoinc.com.au/_rad...,10999,212194,6 Cylinders,Sports Automatic,Diesel,2025-10-24,2009,BMW,212194
143,X,https://www.cheapcarco.com.au/used-vehicles-ro...,https://radius-cdn.ramp.coxautoinc.com.au/_rad...,19999,142313,6 Cylinders,Sports Automatic,Diesel,2025-10-24,2015,BMW,142313
144,XV,https://www.cheapcarco.com.au/used-vehicles-ro...,https://radius-cdn.ramp.coxautoinc.com.au/_rad...,10999,231462,4 Cylinders,Manual,Petrol - Unleaded,2025-10-24,2013,Subaru,231462
